In [40]:
import datajoint as dj
import numpy as np

In [41]:


ta3p100 = dj.create_virtual_module("ta3p100","microns_ta3p100")
ta3 = dj.create_virtual_module("ta3","microns_ta3")
pinky = dj.create_virtual_module("pinky","microns_pinky")
pinky_nda = dj.create_virtual_module("pinky_nda","microns_pinky_nda")

schema = dj.schema("microns_pinky")

In [45]:
pinky.ProofreadLabel()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,author_original name of last editor,author_proofreader name of last editor,date_time the last time it was edited,vertices label data for the vertices,triangles label data for the faces,edges label data for the edges,status the index of the status descriptor that can be references by the StatusKey
3,648518346341371119,0.35,computer_Auto,celiib,2019-06-11 18:52:17,=BLOB=,=BLOB=,=BLOB=,complete
3,648518346349386137,0.35,computer_Auto,celiib,2019-06-11 18:53:38,=BLOB=,=BLOB=,=BLOB=,complete
3,648518346349470171,0.35,computer_Auto,celiib,2019-06-11 18:49:31,=BLOB=,=BLOB=,=BLOB=,complete
3,648518346349471156,0.35,computer_Auto,celiib,2019-06-12 13:53:20,=BLOB=,=BLOB=,=BLOB=,complete
3,648518346349471500,0.35,computer_Auto,celiib,2019-06-12 14:18:35,=BLOB=,=BLOB=,=BLOB=,complete
3,648518346349471562,0.35,computer_Auto,celiib,2019-06-12 14:13:32,=BLOB=,=BLOB=,=BLOB=,complete
3,648518346349471565,0.35,computer_Auto,celiib,2019-06-12 14:11:11,=BLOB=,=BLOB=,=BLOB=,complete


In [44]:
pinky.Decimation35OrphanStitched()

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,n_vertices,n_triangles,vertices,triangles
3,648518346341352891,0.35,21316,42947,=BLOB=,=BLOB=
3,648518346341353058,0.35,38766,77289,=BLOB=,=BLOB=
3,648518346341353574,0.35,58629,117045,=BLOB=,=BLOB=
3,648518346341353607,0.35,68095,135916,=BLOB=,=BLOB=
3,648518346341353788,0.35,9740,19406,=BLOB=,=BLOB=
3,648518346341354313,0.35,45794,91301,=BLOB=,=BLOB=
3,648518346341354496,0.35,74516,148633,=BLOB=,=BLOB=


In [24]:
(pinky.Mesh() & "segmentation=3") - pinky.AllenSoma & ( pinky_nda.Spike() & "segmentation=3") - pinky.Neurite.proj()
    

segmentation segmentation id,segment_id segment id unique within each Segmentation,n_vertices number of vertices in this mesh,n_triangles number of triangles in this mesh,"vertices x,y,z coordinates of vertices",triangles triangles (triplets of vertices)
3,648518346341353058,111754,222198,=BLOB=,=BLOB=
3,648518346341353574,169648,336849,=BLOB=,=BLOB=
3,648518346341354313,131505,261279,=BLOB=,=BLOB=
3,648518346341355198,59818,118694,=BLOB=,=BLOB=
3,648518346341356256,908546,1806745,=BLOB=,=BLOB=
3,648518346341359123,248717,495526,=BLOB=,=BLOB=
3,648518346341360739,278231,553936,=BLOB=,=BLOB=


In [36]:
pinky.OrphanStitchedMeshVp2() & "stitched_addon_perc > 0.01"# & "n_triangles>5000"
pinky.OrphanStitchedMeshVp2() & "n_triangles>1000000"

segmentation segmentation id,segment_id segment id unique within each Segmentation,n_vertices number of vertices in this mesh,n_triangles number of triangles in this mesh,"vertices x,y,z coordinates of vertices",triangles triangles (triplets of vertices),n_pieces number of unconnected mesh pieces outside the largest mesh piece,largest_piece_perc number of faces percentage of largest mesh piece in respect to total mesh,outside_perc number of faces percentage of mesh outside the biggest mesh piece,n_stitched number of mesh pieces stitched back to main mesh,stitched_addon_perc number of faces percentage of pieces that were stitched back in respect to largest mesh piece,n_unstitched number of mesh pieces remaining unstitched back to main mesh,unstitched_perc number of faces percentage of pieces that were not in respect to largest mesh piece
3,648518346341356256,904926,1803313,=BLOB=,=BLOB=,0,0.99810,0.99810,0,0.00000,0,0.00000
3,648518346341361981,728597,1453345,=BLOB=,=BLOB=,0,0.99818,0.99818,0,0.00000,0,0.00000
3,648518346341362995,505807,1008241,=BLOB=,=BLOB=,0,0.99891,0.99891,0,0.00000,0,0.00000
3,648518346341369341,516606,1030257,=BLOB=,=BLOB=,0,0.99923,0.99923,0,0.00000,0,0.00000
3,648518346341373475,575160,1146734,=BLOB=,=BLOB=,0,0.99728,0.99728,0,0.00000,0,0.00000
3,648518346341386441,620793,1238095,=BLOB=,=BLOB=,1,0.99770,0.99773,0,0.00000,1,0.00003
3,648518346341399264,793335,1586347,=BLOB=,=BLOB=,1,0.97498,0.97502,0,0.00000,1,0.00004


In [ ]:
dj.ERD(schema)

In [ ]:
pinky.WholeAutoAnnotations()

In [ ]:
"""
New tables to create

WholeAutoAnnotationsLabel(): where the original labels will need to be
ProofreadLabel(): where the manually proofread lables
LabelKey()




"""

In [ ]:
ta3p100.ProofreadLabel()

In [ ]:
WholeAutoAnnotationsLabel.describe()

In [ ]:
pinky.WholeAutoAnnotations()

In [ ]:
WholeAutoAnnotationsLabel.drop()

In [ ]:
import time
from datetime import datetime
@schema
class WholeAutoAnnotationsLabel(dj.Computed):
    definition = """
    # puts the whole neuron automated labels into a table that is easily plugged into blender plugin
    -> pinky.WholeAutoAnnotations
    author_original      : varchar(20)                  # name of last editor
    author_proofreader   : varchar(20)                  # name of last editor
    date_time=CURRENT_TIMESTAMP : timestamp             # the last time it was edited
    ---
    vertices             : longblob                     # label data for the vertices
    triangles            : longblob                     # label data for the faces
    edges                : longblob                     # label data for the edges
    status               : varchar(16)                  # the index of the status descriptor that can be references by the StatusKey
    
    """
    
    def make(self, key):
        #print("key = " + str(key))
        segment_id = key["segment_id"]
        print(f"Working on {segment_id}")
        #get the vertices and triangle lables
        vertices, triangles = (pinky.WholeAutoAnnotations & key).fetch1("vertices","triangles")
        
        self.insert1(dict(key,
                    author_original="computer_Auto",
                    author_proofreader="bcelii",
                    date_time=str(datetime.now())[0:19],
                    vertices=vertices,
                    triangles=triangles,
                    edges=np.array([]),
                    status="complete"),skip_duplicates=True)
        
        

In [ ]:
start = time.time()
WholeAutoAnnotationsLabel.populate(reserve_jobs=True)
print(time.time() - start)

In [ ]:
(schema.jobs & "table_name='__whole_auto_annotations_label'").delete()

In [ ]:
pinky.PymeshfixDecimatedExcitatoryStitchedMesh

In [ ]:
pinky.WholeAutoAnnotationsLabelClusters3.drop()

In [ ]:
import time
from datetime import datetime
@schema
class WholeAutoAnnotationsLabelClusters3(dj.Computed):
    definition = """
    # puts the whole neuron automated labels into a table that is easily plugged into blender plugin
    -> pinky.WholeAutoAnnotationsClusters3
    author_original      : varchar(20)                  # name of last editor
    author_proofreader   : varchar(20)                  # name of last editor
    date_time=CURRENT_TIMESTAMP : timestamp             # the last time it was edited
    ---
    vertices             : longblob                     # label data for the vertices
    triangles            : longblob                     # label data for the faces
    edges                : longblob                     # label data for the edges
    status               : varchar(16)                  # the index of the status descriptor that can be references by the StatusKey
    
    """
    
    def make(self, key):
        #print("key = " + str(key))
        segment_id = key["segment_id"]
        print(f"Working on {segment_id}")
        #get the vertices and triangle lables
        vertices, triangles = (pinky.WholeAutoAnnotationsClusters3 & key).fetch1("vertices","triangles")
        
        self.insert1(dict(key,
                    author_original="computer_Auto",
                    author_proofreader="bcelii",
                    date_time=str(datetime.now())[0:19],
                    vertices=vertices,
                    triangles=triangles,
                    edges=np.array([]),
                    status="complete"),skip_duplicates=True)
        
        

In [ ]:
start = time.time()
WholeAutoAnnotationsLabelClusters3.populate(reserve_jobs=True)
print(time.time() - start)

In [ ]:
pinky.WholeAutoAnnotationsLabel()

In [ ]:
ta3p100.ProofreadLabelFinal.describe()

# Make the final proofread Label table

In [ ]:
pinky.PymeshfixDecimatedExcitatoryStitchedMesh

In [ ]:
pinky.ProofreadLabel.drop()

In [ ]:
@schema
class ProofreadLabel(dj.Manual):
    definition="""
    -> pinky.PymeshfixDecimatedExcitatoryStitchedMesh
    author_original      : varchar(20)                  # name of last editor
    author_proofreader   : varchar(20)                  # name of last editor
    date_time=CURRENT_TIMESTAMP : timestamp                    # the last time it was edited
    ---
    vertices             : longblob                     # label data for the vertices
    triangles            : longblob                     # label data for the faces
    edges                : longblob                     # label data for the edges
    status               : varchar(16)                  # the index of the status descriptor that can be references by the StatusKey

    """

In [ ]:
pinky.ProofreadLabel()

# Make the author and label tables

In [ ]:
ta3p100.LabelKey()
and / or 
ta3.LabelKey2()


In [ ]:
pinky.LabelKey.drop()

In [ ]:
@schema
class LabelKey(dj.Lookup):
    definition="""
    # maps numeric labels to descriptive and color labels
    numeric              : tinyint unsigned             # numeric label of the compartment
    ---
    description          : varchar(16)                  # descriptive name of the label
    color                : varchar(16)                  # representative color of the compartment
    blender_colors       : tinyblob                     # color RGB values to be used in blender
    """

In [ ]:
ta3.LabelKey2.describe()

In [ ]:
LabelKey.insert(ta3p100.LabelKey())

In [ ]:
pinky.LabelKey().fetch()

In [ ]:
total_old_labels = ta3.LabelKey2().fetch(as_dict=True)
total_old_labels

In [ ]:
# (17, 'shaft', 'dark orange'),
#        (18, 'non-spine', 'light orange')],
total_old_labels.append(dict(numeric=17,
                             description="shaft",
                             color="dark orange",
                            blender_colors=np.array([0.8,0.186,0])))


In [ ]:
total_old_labels.append(dict(numeric=18,
                             description="non-spine",
                             color="light orange",
                            blender_colors=np.array([0.8,0.382,0])))

In [ ]:
pinky.LabelKey.insert(total_old_labels)

In [ ]:
pinky.Authors()

In [ ]:
ta3.AuthorsLast.describe()

In [ ]:
@schema
class Authors(dj.Manual):
    definition = """
    # maps numeric labels to descriptive labels
    username             : varchar(20)                  # username the person pcisk
    ---
    real_name            : varchar(40)                  # the real name of that corresponds to the username
    """

In [ ]:
pinky.Authors()

In [ ]:
pinky.PymeshfixDecimatedExcitatoryStitchedMesh()

In [ ]:
pinky.WholeAutoAnnotationsLabel()

In [ ]:
pinky.Decimation35ExcitatoryStitchedMesh()

In [ ]:
pinky.PymeshfixDecimatedExcitatoryStitchedMesh()

In [ ]:
(schema.jobs & "table_name='__whole_auto_annotations'").delete()

In [ ]:
pinky.WholeAutoAnnotationsClusters3()

In [ ]:
pinky.ProofreadLabel() #& "date_time>'2019-06-13 00:00:00'"

In [ ]:
ta3p100.Contact()

In [ ]:
ta3p100.ContactTest()